In [5]:
import google.generativeai as genai
import os

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

d:\Github\AI-Summary-Paper-Tool\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
import chromadb
import numpy as np
from scipy.spatial.distance import cosine

# Initialize Chroma client
client = chromadb.PersistentClient(path="chroma_db")
collection = client.get_or_create_collection(name="paper_summaries")

def get_closest_document(query):
    # Step 1: Embed the query
    result = genai.embed_content(
        model="models/text-embedding-004",
        content=query,
    )
    
    query_embedding = result['embedding']
    
    results = collection.get(include=["embeddings", "documents", "metadatas"])
    embeddings = results["embeddings"]
    documents = results["documents"]
    metadatas = results["metadatas"]
    
    if embeddings is None or len(embeddings) == 0:
        return {"error": "No documents found in the database."}
    
    similarities = []
    for embedding in embeddings:
        embedding = np.array(embedding) if isinstance(embedding, list) else embedding
        similarity = 1 - cosine(query_embedding, embedding) # Compute cosine similarity (1 - cosine distance)
        similarities.append(similarity)
    
    closest_index = np.argmax(similarities)  # Index of the highest similarity score
    closest_similarity = similarities[closest_index]
    
    # Step 5: Extract the closest document's details
    closest_document = {
        "title": metadatas[closest_index].get("title", "No title available"),
        "content": documents[closest_index],
        "similarity_score": closest_similarity
    }
    
    return closest_document

In [10]:
def format_and_display_result(closest_doc):
    if "error" in closest_doc:
        print(closest_doc["error"])
        return
    
    title = closest_doc.get("title", "No title available")
    content = closest_doc.get("content", "No content available")
    similarity_score = closest_doc.get("similarity_score", 0.0)
    
    # Print formatted output
    print("\nClosest Document:")
    print("=" * 50)
    print(f"Title: {title}")
    print("-" * 50)
    print(f"Similarity Score: {similarity_score:.4f}")
    print("-" * 50)
    print("Content:")
    print(content)
    print("=" * 50)

In [13]:
query = "I want to learn about differentially private machine learning algorithms."
result = get_closest_document(query)

format_and_display_result(result)


Closest Document:
Title: Differentially Private Attention Computation
--------------------------------------------------
Similarity Score: 0.6847
--------------------------------------------------
Content:
Here is a summary of the research paper, addressing your specific questions and requirements:

**Summary of "Differentially Private Attention Computation"**

This research paper tackles the critical problem of privacy in large language models (LLMs), specifically concerning the attention mechanism, a core component of Transformer architectures. The authors identify that LLMs can inadvertently leak sensitive information present in training data or user inputs during inference. To mitigate this risk, the paper proposes a novel and efficient algorithm for approximating the attention matrix while providing provable differential privacy (DP) guarantees.

**1. Research Problem:**

*   **Main Objective:** The primary objective is to develop an algorithm that computes the attention matrix i